In [1]:
#importing what is needed
import time
import pandas as pd
import requests
import json5


In [8]:
url_base = 'https://www.indeed.com/jobs?q=machine%20learning&l=remote&explvl=entry_level&fromage=1&limit=50'

In [9]:
jobs = []

In [10]:
err_count = 0
text = ''
r = requests.get(url_base)
while r.status_code!=200 & err_count < 5:
    time.sleep(5)
    err_count += 1
    r = requests.get(url_base)
if err_count < 5:
    text = r.text
for i in range(text.count('jobmap[')):
    start = text.find('jobmap[{}]'.format(i))
    if start==-1:
        continue
    end = text.find('jobmap[{}]'.format(i+1))
    s = text.find('{',start)
    if end == -1:
        e = text.find('}',s)
    else:
        e = text.rfind('}',s,end)
    job = json5.loads(text[s:e+1])
    jobs.append(job)
   